In [4]:
import os
import re
import json
import time
import boto3
import random
import requests
import base64
import sagemaker
import pandas as pd

from pprint import pprint
from IPython.display import Image 
from sagemaker import ModelPackage
from sagemaker import get_execution_role

## Define Cell Variables and Sagemaker Sessions

In [5]:
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
account_id = sagemaker_session.account_id()

role = get_execution_role()
model_package_arn = "arn:aws:sagemaker:us-east-1:628804397842:model-package/snap2place-2021-12-16"
model_name= "snap2place-2021-12-16"

real_time_inference_instance_type = "ml.m5d.4xlarge"
batch_transform_inference_instance_type = "ml.m5.4xlarge"
batch_transform_inference_instance_count = 10

content_type = "application/json"

## Create Endpoint

In [7]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----!

## Run Inference with single point

In [8]:
# Example payload
payload = '{"lat":33.89382700435817,"lng":-118.36993614211679}'

In [9]:
client = boto3.client('sagemaker-runtime', region_name='us-east-1' )

response = client.invoke_endpoint(
            EndpointName=model_name,
            ContentType=content_type,
            Accept=content_type,
            Body=payload)

print(response['Body'].read().decode('utf-8'))

{"venues":[["582378fec4c9fa2d07af3a81",0.3463153380197817],["4b0dc578f964a520c94f23e3",0.2756845236101784],["575064be498e851d7d384392",0.21033035875488454],["5dcf240eb4b85f0007f0b866",0.11044341115771941],["4fb260c7e4b0da11f3368411",0.019192698486826257],["5d8d530cd2219e000806d96b",0.0129994458361149],["4becaae88bbcc92876a88cb1",0.012861944704440053],["511d07f6e4b0113e087dbb38",0.012172279430054634]]}


In [10]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## Run Batch Inference

In [11]:
transformer = model.transformer(1, batch_transform_inference_instance_type)

In [26]:
# Run a batch-transform job
input_data_path = "s3://4sq-product-vpc/sagemaker/v=00002/batch_data/tiny_input/test.json"
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(input_data_path, content_type=content_type)
transformer.wait()

.........................................2021-12-19T22:43:05.403:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
22:43:01,709 |-INFO in ch.qos.logback.classic.LoggerContext[default] - Found resource [/opt/ml/code/logback.xml] at [file:/opt/ml/code/logback.xml]
22:43:01,773 |-INFO in ch.qos.logback.classic.joran.action.ConfigurationAction - debug attribute not set
22:43:01,788 |-ERROR in ch.qos.logback.core.joran.spi.Interpreter@2:65 - no applicable action for [filter], current ElementPath  is [[configuration][filter]]
22:43:01,788 |-ERROR in ch.qos.logback.core.joran.spi.Interpreter@3:12 - no applicable action for [level], current ElementPath  is [[configuration][filter][level]]
22:43:01,788 |-INFO in ch.qos.logback.classic.joran.action.ConfigurationAction - End of configuration.
22:43:01,790 |-INFO in ch.qos.logback.classic.joran.JoranConfigurator@70a9f84e - Registering current configuration as safe fallback point
Venue data count: 760
S2 data

In [27]:
from urllib.parse import urlparse
parsed_url = urlparse(transformer.output_path)
bucket_name = parsed_url.netloc
file_key = "{}/{}.out".format(parsed_url.path[1:], input_data_path.split("/")[-1])
s3_client = sagemaker_session.boto_session.client("s3")

response = s3_client.get_object(Bucket=sagemaker_session.default_bucket(), Key=file_key)
print(response)

{'ResponseMetadata': {'RequestId': 'H9V1F5246PM3F99Z', 'HostId': 'Mipz7Zw8mhmbkM1rej5B3cgNWfN4ijT9UGEPUwlwRLPvwXbvckEE2LCx9K74uSYGY8TkV16KbKs=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Mipz7Zw8mhmbkM1rej5B3cgNWfN4ijT9UGEPUwlwRLPvwXbvckEE2LCx9K74uSYGY8TkV16KbKs=', 'x-amz-request-id': 'H9V1F5246PM3F99Z', 'date': 'Sun, 19 Dec 2021 22:43:40 GMT', 'last-modified': 'Sun, 19 Dec 2021 22:43:08 GMT', 'etag': '"80ca3c412996308d2d7d97bcc5ebc291"', 'x-amz-server-side-encryption': 'aws:kms', 'x-amz-server-side-encryption-aws-kms-key-id': 'arn:aws:kms:us-east-1:628804397842:key/e68dbd41-d966-4643-81e2-14d54baf9156', 'accept-ranges': 'bytes', 'content-type': 'application/octet-stream', 'server': 'AmazonS3', 'content-length': '108'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2021, 12, 19, 22, 43, 8, tzinfo=tzutc()), 'ContentLength': 108, 'ETag': '"80ca3c412996308d2d7d97bcc5ebc291"', 'ContentType': 'application/octet-stream', 'ServerSideEncryption': '